<a href="https://colab.research.google.com/github/pjsandwich/bda-718-group-1/blob/master/Unifiedcodebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark
pip install seaborn

  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=a65bd487b4223d7ea56575d3527aab61d86ecf9bf4e70bc5252c3e6d25afc185
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml import feature
import re
from pyspark.sql import functions as fn
from pyspark.sql.functions import col, regexp_replace, split
from pyspark.sql.utils import AnalysisException
from pyspark.sql import Row
from pyspark.sql.types import IntegerType, DateType, FloatType, BooleanType
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

spark = SparkSession \
  .builder \
  .master("local[*]")\
  .config("spark.memory.fraction", 0.8) \
  .config("spark.executor.memory", "12g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.memory.offHeap.enabled",'true')\
  .config("spark.memory.offHeap.size","12g")\
  .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql.functions import col, udf
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import create_map, lit
from itertools import chain
from pyspark.mllib import numpy as np
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.tuning import ParamGridBuilder

Mounted at /content/drive


In [4]:
new_df = spark.read.format("csv").option("header", "true").option('inferSchema','true').load("drive/Shared drives/IST 718 Group1/spaghetti.csv")

In [5]:
new_df.show(5)

+------------+----+---------------+-----+-------------+-----------------+------------+----------+----------------+-------------------+-----------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------+-----+------+------+---------+--------+------+-----------+-------------+---------+--------------------+----------+---------------+-------+-----------------+-----------+-------+-------+--------------+-------------+------+--------------------+-----------+------------+--------------------+------------+---------+-----+----+-----------+----------+------------+-----------+-------------+-----------+--------------+--------------+------+-----------+------------+----------------+---------+----------------+------------+---------------+------+----+----+-----+---+------+---+---+---+----+----+----+----+----+-----+----+----+----+----+-------+-----+--------+----+--------+----+-------+----+------+----

In [6]:
new_df.describe().show()

+-------+------------------+-----+-----------+-----------+-----------+------------------+------------------+------------------+------------------+-------------------+-----------+--------------+------------------+------------------+---------+-----------------+--------------------+------------------+--------------------+------------------+------------------+-----------+-------------+------------------+--------------------+----------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+--------------------+------------+--------------------+------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+--------

In [7]:
new_df.printSchema()

root
 |-- back_legroom: double (nullable = true)
 |-- bed: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: double (nullable = true)
 |-- daysonmarket: integer (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- engine_cylinders: integer (nullable = true)
 |-- engine_displacement: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- fleet: boolean (nullable = true)
 |-- frame_damaged: boolean (nullable = true)
 |-- franchise_dealer: boolean (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: double (nullable = true)
 |-- fuel_tank_volume: double (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- has_accidents: boolean (nullable = true)
 |-- height: double (nullable = true)
 |-- highway_fuel_economy: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- interior_color: string (n